In [ ]:
import time
from collections import OrderedDict
import numpy as np
import nidaqmx
import pandas as pd

import quantities as pq
from nidaqmx.stream_writers import (
    DigitalSingleChannelWriter, DigitalMultiChannelWriter)
from nidaqmx.utils import flatten_channel_string
from nidaqmx.constants import (
    LineGrouping, AcquisitionType, DigitalWidthUnits, Edge,
    HandshakeStartCondition, Level, MIOAIConvertTimebaseSource,
    OverflowBehavior, TaskMode, Polarity, RegenerationMode,
    SampleInputDataWhen, SampleTimingType, UnderflowBehavior)
from nidaqmx.error_codes import DAQmxErrors, DAQmxWarnings
from nidaqmx.errors import (
    check_for_error, is_string_buffer_too_small, DaqError, DaqResourceWarning)

from scipy.optimize import fsolve, least_squares

from olfactometer.PID_reader import PID_Tester
from olfactometer.valve_driver import ValveDriver
from olfactometer.equipment import Olfactometer
from olfactometer.my_equipment import MyValve, MyJar, MyLowMFC, \
                                      MyMediumMFC, MyHighMFC
from olfactometer.odorants import Solution, Compound, ChemicalOrder, \
                                  Vendor, Molecule
# from graph import make_graph, draw_graph
from pprint import pprint
from olfactometer.smell_engine import SmellEngine
from olfactometer.data_container import DataContainer
from olfactometer.ui import UI
from IPython.display import display

np.set_printoptions(precision=6)

In [ ]:
# Initialize # molecules
#molecules = OrderedDict([(702, 'Ethanol')])
molecules = OrderedDict([(7410, 'Acetophenone'), (7439, 'carvone'), (440917, 'd-limonene')])
# Initialize UI
ui = UI(molecules, print_PID_average=True)
NUM_PID_SAMPLES = 10000

In [ ]:
eth = "./ethanol_11-2-21.pkl"
odors = "./odors.pkl"

# To run without odor table
smell_engine = SmellEngine(1000, len(molecules), DataContainer(), debug_mode=False, write_flag=False, PID_mode = True,
                                       look_up_table_path=None, oms=molecules)
# To run with odor table, diff being specification of pkl file.
# smell_engine = SmellEngine(1000, len(molecules), DataContainer(), debug_mode=False, write_flag=False, PID_mode = True,
#                                        look_up_table_path=odors, oms=molecules)

# Initialize system
smell_engine.set_odorant_molecule_ids(list(molecules.keys()))
smell_engine.set_odorant_molecule_dilutions([10,1,10])
smell_engine.initialize_smell_engine_system()
smell_engine.smell_controller.valve_driver.num_pid_samples = NUM_PID_SAMPLES
smell_engine.olfactometer.loaded_molecules

In [ ]:
# Tell me how to get max_flow_rates and n_jars from smell_engine
n_jars = len(smell_engine.olfactometer.jars)
print(n_jars)
max_flow_rates = smell_engine.smell_controller.get_max_flow_rates()
total_vapor = smell_engine.smell_controller.get_vapor_concs_dense(set(list(smell_engine.target_concentration))).sum(axis=0)
print(f"Max Flow Rate {max_flow_rates}\nVapor Pressures {total_vapor}")

In [ ]:
##### INITIALIZE CONCENTRATION SLIDERS
ui.odorConcentrationUI()

In [ ]:
# SPECIFYING CONCENTRATIONS WITHOUT THREADED SLIDERS
concentration_mixtures = ui.odorConcentrationValues() # Read in user-specified concentrations
#     print(concentration_mixtures)
smell_engine.set_desired_concentrations(concentration_mixtures)  # Assign target concentrationskeys

In [ ]:
ui.dutyCyclesUI()

In [ ]:
ui.mfcUI()

In [ ]:
smell_engine.set_valve_duty_cycles(ui.dutyCyclesValues())
print(ui.dutyCyclesValues())
mfcs = []  # TODO: Fix referencing order of MFC setpoints to be automatic
mfcs.append(ui.mfcValues()[0])
mfcs.append(ui.mfcValues()[2])
mfcs.append(ui.mfcValues()[1])
smell_engine.set_mfc_setpoints(ui.mfcValues())

In [ ]:
# Timer setup specifies sampling frequency, sampling rate specifics # of samples to read
PID_mode = True
pid = PID_Tester(ui, smell_engine, PID_mode, cont_read_conc=False,sampling_rate = NUM_PID_SAMPLES)

pid.timer_setup(.00001)
pid.timer_start()
display(ui.timeSeries)

In [ ]:
# %matplotlib widget
smell_engine.close_smell_engine()